In [1]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['font.size']=16

In [2]:
import numpy 
from matplotlib import pyplot, cm 
from mpl_toolkits.mplot3d import Axes3D 

 
def L2_rel_error(p, pn):
    return numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2)) 

 
def plot_3D(x, y, p, elev=30, azi=45):
    fig = pyplot.figure(figsize=(11,7), dpi=100) 
    ax = fig.gca(projection='3d') 
    X,Y = numpy.meshgrid(x,y) 
    surf = ax.plot_surface(X,Y,p[:], rstride=1, cstride=1, cmap=cm.viridis, 
    linewidth=0, antialiased=False) 
    ax.set_xlabel('$x$') 
    ax.set_ylabel('$y$') 
    ax.set_zlabel('$z$') 
    ax.view_init(elev,azi) 
 
 
def p_analytical(x, y):
    X,Y = numpy.meshgrid(x,y) 
    pxy = numpy.sinh(1.5*numpy.pi*Y / x[-1]) /(numpy.sinh(1.5*numpy.pi*y[-1]/x[-1]))*numpy.sin(1.5*numpy.pi*X/x[-1]) 
    return pxy 

In [3]:
#initialize the grid space

nx=128
ny=128

L=5
H=5

x=numpy.linspace(0,L,nx)
y=numpy.linspace(0,H,ny)

dx=L/(nx-1)
dy=H/(ny-1)

p0=numpy.zeros((ny,nx))

p0[-1,:]=numpy.sin(1.5*numpy.pi*x/x[-1])

In [4]:
def laplace2d(p, target):
    
    norm=1
    
    pn=numpy.empty_like(p)
    
    interations=0
    
    while norm > target:
        
        pn=p.copy()
        p[1:-1,1:-1]= 0.25*(pn[1:-1,2:] + pn[1:-1, :-2] + pn[2:,1:-1] + pn[:-2, 1:-1])
        
        #Nuemann BC
        p[1:-1, -1]=0.25*(2*pn[1:-1,-2]+pn[2:,-1] +pn[:-2, -1])
        
        norm = numpy.sqrt(numpy.sum(p-pn)**2/numpy.sum(pn**2))
        interations+=1
        
    return p, interations

In [5]:
target=1e-8

In [6]:
p, iterations=laplace2d(p0.copy(), target)

In [7]:
print("Jacobi method took {} interations at toleranace {}".format(iterations, target))

Jacobi method took 27057 interations at toleranace 1e-08


In [8]:
%%timeit
laplace2d(p0.copy(), target)

1 loops, best of 3: 4.21 s per loop


In [9]:
pan=p_analytical(x, y)

In [10]:
L2_rel_error(p, pan)

7.7434413870690639e-05

In [11]:
def laplace2d_gauss_seidel(p, nx, ny, target):
    
    iterations=0
    iter_diff=target+1
    
    while iter_diff>target:
        
        pn=p.copy()
        iter_diff=0
        
        for j in range(1, ny-1):
            
            for i in range(1, nx-1):
                
                p[j,i]=0.25*(p[j,i-1]+p[j,i+1]+p[j-1,i]+p[j+1,i])
                
                iter_diff+=(p[j,i]-pn[j,i])**2
                
        #Nuemann BC
        for j in range(1, ny-1):
            
            p[j,-1]=.25*(2*p[j,-2]+p[j+1,-1]+p[j-1,-1])
            
        iter_diff=numpy.sqrt(iter_diff/numpy.sum(pn**2))
        iterations+=1
        
    return p, iterations

In [12]:
import numba
from numba import jit

In [13]:
def fib_it(n):
    
    a=1
    b=1
    for i in range(n-2):
        a, b = b , a+b
        
    return b

In [14]:
%%timeit
fib_it(500000)

1 loops, best of 3: 2.64 s per loop


In [15]:
@jit
def fib_it(n):
    
    a=1
    b=1
    for i in range(n-2):
        a, b = b , a+b
        
    return b

In [16]:
%%timeit
fib_it(500000)

The slowest run took 210.74 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 245 µs per loop


In [17]:
%%timeit
fib_it(500000)

1000 loops, best of 3: 245 µs per loop


@jit(nopython=True)

In [18]:
@jit(nopython=True)
def laplace2d_jacobi(p, pn, l2_target):
    '''Solves the Laplace equation using the Jacobi method 
    with a 5-point stencil
    
    Parameters:
    ----------
    p: 2D array of float
        Initial potential distribution
    pn: 2D array of float
        Allocated array for previous potential distribution
    l2_target: float
        Stopping criterion
        
    Returns:
    -------
    p: 2D array of float
        Potential distribution after relaxation
    '''
        
    iterations = 0
    iter_diff = l2_target+1 #init iter_diff to be larger than l2_target
    denominator = 0.0
    ny, nx = p.shape
    l2_diff = numpy.zeros(20000)
    
    while iter_diff > l2_target:
        for j in range(ny):
            for i in range(nx):
                pn[j,i] = p[j,i]
                
        iter_diff = 0.0
        denominator = 0.0
        
        for j in range(1,ny-1):
            for i in range(1,nx-1):
                p[j,i] = .25 * (pn[j,i-1] + pn[j,i+1] + pn[j-1,i] + pn[j+1,i])
                
        
        #Neumann 2nd-order BC
        for j in range(1,ny-1):
            p[j,-1] = .25 * (2*pn[j,-2] + pn[j+1,-1] + pn[j-1, -1])
            
            
        for j in range(ny):
            for i in range(nx):
                iter_diff += (p[j,i] - pn[j,i])**2
                denominator += (pn[j,i]*pn[j,i])
        
        
        iter_diff /= denominator
        iter_diff = iter_diff**0.5
        l2_diff[iterations] = iter_diff
        iterations += 1    
        
    return p, iterations, l2_diff

In [22]:
p, iterations, diffj=laplace2d_jacobi(p0.copy(), p0.copy(), 1e-8)
print("Numba Jacobi method took {} iterations at tolerance {}".format(iterations, target))

Numba Jacobi method took 19993 iterations at tolerance 1e-08


In [23]:
%%timeit 
laplace2d_jacobi(p0.copy(), p0.copy(), 1e-8)

1 loops, best of 3: 704 ms per loop


In [24]:




@jit(nopython=True)
def laplace2d_gauss_seidel(p, pn, l2_target):
    '''Solves the Laplace equation using Gauss-Seidel method 
    with a 5-point stencil
    
    Parameters:
    ----------
    p: 2D array of float
        Initial potential distribution
    pn: 2D array of float
        Allocated array for previous potential distribution
    l2_target: float
        Stopping criterion
        
    Returns:
    -------
    p: 2D array of float
        Potential distribution after relaxation
    '''
    
    iterations = 0
    iter_diff = l2_target + 1 #initialize iter_diff to be larger than l2_target
    denominator = 0.0
    ny, nx = p.shape
    l2_diff = numpy.zeros(20000)
    
    while iter_diff > l2_target:
        for j in range(ny):
            for i in range(nx):
                pn[j,i] = p[j,i]
                
        iter_diff = 0.0
        denominator = 0.0
        
        for j in range(1,ny-1):
            for i in range(1,nx-1):
                p[j,i] = .25 * (p[j,i-1] + p[j,i+1] + p[j-1,i] + p[j+1,i])
                
        
        #Neumann 2nd-order BC
        for j in range(1,ny-1):
            p[j,-1] = .25 * (2*p[j,-2] + p[j+1,-1] + p[j-1, -1])
            
            
        for j in range(ny):
            for i in range(nx):
                iter_diff += (p[j,i] - pn[j,i])**2
                denominator += (pn[j,i]*pn[j,i])
        
        
        iter_diff /= denominator
        iter_diff = iter_diff**0.5
        l2_diff[iterations] = iter_diff
        iterations += 1    
        
    return p, iterations, l2_diff


In [26]:

p, iterations, l2_diffGS = laplace2d_gauss_seidel(p0.copy(), p0.copy(), 1e-8)

print("Numba Gauss-Seidel method took {} iterations at tolerance {}".format(iterations, target))


Numba Gauss-Seidel method took 13939 iterations at tolerance 1e-08


In [27]:
%%timeit
laplace2d_gauss_seidel(p0.copy(), p0.copy(), 1e-8)

1 loops, best of 3: 1.69 s per loop


In [28]:
@jit(nopython=True)
def laplace2d_SOR(p, pn, l2_target, omega):
    '''Solves the Laplace equation using SOR with a 5-point stencil
    
    Parameters:
    ----------
    p: 2D array of float
        Initial potential distribution
    pn: 2D array of float
        Allocated array for previous potential distribution
    l2_target: float
        Stopping criterion
    omega: float
        Relaxation parameter
        
    Returns:
    -------
    p: 2D array of float
        Potential distribution after relaxation
    '''    
    
    iterations = 0
    iter_diff = l2_target + 1 #initialize iter_diff to be larger than l2_target
    denominator = 0.0
    ny, nx = p.shape
    l2_diff = numpy.zeros(20000)
    
    while iter_diff > l2_target:
        for j in range(ny):
            for i in range(nx):
                pn[j,i] = p[j,i]
                
        iter_diff = 0.0
        denominator = 0.0
        
        for j in range(1,ny-1):
            for i in range(1,nx-1):
                p[j,i] = (1-omega)*p[j,i] + omega*.25 * (p[j,i-1] + p[j,i+1] + p[j-1,i] + p[j+1,i])
        
        #Neumann 2nd-order BC
        for j in range(1,ny-1):
            p[j,-1] = .25 * (2*p[j,-2] + p[j+1,-1] + p[j-1, -1])
            
        for j in range(ny):
            for i in range(nx):
                iter_diff += (p[j,i] - pn[j,i])**2
                denominator += (pn[j,i]*pn[j,i])
        
        
        iter_diff /= denominator
        iter_diff = iter_diff**0.5
        l2_diff[iterations] = iter_diff
        iterations += 1
        
    
    return p, iterations, l2_diff

In [29]:
l2_target = 1e-8
omega = 1
p, iterations, l2_diffSOR = laplace2d_SOR(p0.copy(), p0.copy(), l2_target, omega)

print("Numba SOR method took {} iterations\
 at tolerance {} with omega = {}".format(iterations, l2_target, omega))

Numba SOR method took 13939 iterations at tolerance 1e-08 with omega = 1


In [30]:
l2_target = 1e-8
omega = 1.5
p, iterations, l2_diffSOR = laplace2d_SOR(p0.copy(), p0.copy(), l2_target, omega)

print("Numba SOR method took {} iterations\
 at tolerance {} with omega = {}".format(iterations, l2_target, omega))

Numba SOR method took 7108 iterations at tolerance 1e-08 with omega = 1.5


In [31]:
%%timeit
laplace2d_SOR(p0.copy(), p0.copy(), l2_target, omega)

1 loops, best of 3: 990 ms per loop


In [32]:
l2_target = 1e-8
omega = 2./(1 + numpy.pi/nx)
p, iterations, l2_diffSORopt = laplace2d_SOR(p0.copy(), p0.copy(), l2_target, omega)

print("Numba SOR method took {} iterations\
 at tolerance {} with omega = {:.4f}".format(iterations, l2_target, omega))

Numba SOR method took 1110 iterations at tolerance 1e-08 with omega = 1.9521


In [33]:
%%timeit
laplace2d_SOR(p0.copy(), p0.copy(), l2_target, omega)

10 loops, best of 3: 157 ms per loop


In [34]:
L2_rel_error(p,pan)

7.7927433550445207e-05

In [37]:
pyplot.figure(figsize=(8,8))
pyplot.xlabel(r'iterations', fontsize=18)
pyplot.ylabel(r'$L_2$-norm', fontsize=18)
pyplot.semilogy(numpy.trim_zeros(12_diffJ,'b'),
                'k-', lw=2, label='Jacobi')
pyplot.semilogy(numpy.trim_zeros(l2_diffGS,'b'), 
                'k--', lw=2, label='Gauss-Seidel')
pyplot.semilogy(numpy.trim_zeros(l2_diffSOR,'b'), 
                'g-', lw=2, label='SOR')
pyplot.semilogy(numpy.trim_zeros(l2_diffSORopt,'b'), 
                'g--', lw=2, label='Optimized SOR')
pyplot.legend(fontsize=16);

SyntaxError: invalid syntax (<ipython-input-37-79a44fb49001>, line 4)